In [70]:
import requests
from lxml import etree
import re
import time
import csv

In [60]:
def crawl_single_page(start:int=0):
    url = f'https://movie.douban.com/review/best/?start={start}'
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"}
    response = requests.get(url, headers=headers)
    tree = etree.HTML(response.content.decode())
    
    for item in tree.xpath('//div[@class="review-list chart "]/div/div[@class="main review-item"]'):
        cover = item.xpath('./a/img/@src')[0]
        title = item.xpath('./a/img/@title')[0]
        comment = ' '.join([line.strip().replace('\xa0', ' ') for line in item.xpath('.//div[@class="short-content"]/text()')]).replace('( )','').strip()
        review_page = item.xpath('./div[@class="main-bd"]/h2/a/@href')[0]
        
        review_resp = requests.get(review_page, headers=headers)
        review_tree = etree.HTML(review_resp.content.decode())
        full_comment = ' '.join([t.strip() for t in review_tree.xpath('//div[@class="review-content clearfix"]//text()')]).strip()
        
        yield cover, title, comment, full_comment

In [77]:
url = f'https://movie.douban.com/review/best/?start=0'
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"}
response = requests.get(url, headers=headers)
tree = etree.HTML(response.content.decode())

# 多页
f = open('豆瓣影评2.csv', 'w', encoding='utf-8', newline='')
csv_writer = csv.writer(f)

max_entries_num = int(re.search('\d+', tree.xpath('//div[@class="paginator"]/span[@class="count"]/text()')[0]).group())
for men in range(0, max_entries_num, 10):
    for info in crawl_single_page(men):
        csv_writer.writerow(info)
    time.sleep(1)
    
f.close()

In [76]:
# 单页
with open('豆瓣影评1.csv', 'w', encoding='utf-8', newline='') as f:
    csv_writer = csv.writer(f)
    for info in crawl_single_page(0):
        csv_writer.writerow(info)